In [62]:
import mysql.connector
from tqdm import tqdm

In [55]:
database = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
)
cursor = database.cursor()

In [56]:
cursor.execute("CREATE DATABASE IF NOT EXISTS nlp_thesis_similarity")
cursor.execute("USE nlp_thesis_similarity")

In [57]:
# Create the main papers table
create_papers_table = """
CREATE TABLE IF NOT EXISTS dewey_papers (
    id VARCHAR(50) PRIMARY KEY,
    title TEXT NOT NULL,
    abstract TEXT,
    publisher VARCHAR(255),
    language VARCHAR(50),
    theme VARCHAR(255),
    category VARCHAR(255),
    sub_category VARCHAR(255),
    source TEXT
)
"""
cursor.execute(create_papers_table)
database.commit()

In [58]:
# Create table for creators (authors)
create_creators_table = """
CREATE TABLE IF NOT EXISTS creators (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    UNIQUE(name)
)
"""
cursor.execute(create_creators_table)

# Create table for paper-creator relationships
create_paper_creators_table = """
CREATE TABLE IF NOT EXISTS paper_creators (
    paper_id VARCHAR(50),
    creator_id INT,
    PRIMARY KEY (paper_id, creator_id),
    FOREIGN KEY (paper_id) REFERENCES dewey_papers(id),
    FOREIGN KEY (creator_id) REFERENCES creators(id)
)
"""
cursor.execute(create_paper_creators_table)
database.commit()

In [59]:
# Create table for contributors (advisors, committee members)
create_contributors_table = """
CREATE TABLE IF NOT EXISTS contributors (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    UNIQUE(name)
)
"""
cursor.execute(create_contributors_table)

# Create table for paper-contributor relationships with role information
create_paper_contributors_table = """
CREATE TABLE IF NOT EXISTS paper_contributors (
    paper_id VARCHAR(50),
    contributor_id INT,
    role VARCHAR(100),
    PRIMARY KEY (paper_id, contributor_id, role),
    FOREIGN KEY (paper_id) REFERENCES dewey_papers(id),
    FOREIGN KEY (contributor_id) REFERENCES contributors(id)
)
"""
cursor.execute(create_paper_contributors_table)
database.commit()

In [60]:
# Create table for subjects/keywords
create_subjects_table = """
CREATE TABLE IF NOT EXISTS subjects (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    UNIQUE(name)
)
"""
cursor.execute(create_subjects_table)

# Create table for paper-subject relationships
create_paper_subjects_table = """
CREATE TABLE IF NOT EXISTS paper_subjects (
    paper_id VARCHAR(50),
    subject_id INT,
    PRIMARY KEY (paper_id, subject_id),
    FOREIGN KEY (paper_id) REFERENCES dewey_papers(id),
    FOREIGN KEY (subject_id) REFERENCES subjects(id)
)
"""
cursor.execute(create_paper_subjects_table)
database.commit()

# Database Schema for NLP Thesis Similarity Analysis

![Database ERD](./erd_sql.png)

## Database Structure
This notebook sets up a MySQL database for storing and analyzing academic papers. The schema includes:

- **dewey_papers**: Stores the main paper information including title, abstract, publisher, and categorization
- **creators**: Authors of the papers
- **contributors**: Advisors and committee members with their roles
- **subjects**: Keywords and subject areas related to papers

The design uses junction tables (paper_creators, paper_contributors, paper_subjects) to establish many-to-many relationships between entities while maintaining data integrity through foreign key constraints.

The database supports importing thesis data from JSON files, with functions to handle both individual and bulk imports from the dewey_thesis directory.

In [61]:
# Example of how to insert data from a JSON file
import json
import os

# Function to insert a paper from a JSON file
def insert_paper_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        paper_data = json.load(file)
    
    # Insert into dewey_papers table
    insert_paper = """
    INSERT INTO dewey_papers (id, title, abstract, publisher, language, theme, category, sub_category, source)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        title = VALUES(title),
        abstract = VALUES(abstract),
        publisher = VALUES(publisher),
        language = VALUES(language),
        theme = VALUES(theme),
        category = VALUES(category),
        sub_category = VALUES(sub_category),
        source = VALUES(source)
    """
    
    paper_values = (
        paper_data.get('id'),
        paper_data.get('title'),
        paper_data.get('abstract'),
        paper_data.get('publisher'),
        paper_data.get('language'),
        paper_data.get('theme'),
        paper_data.get('category'),
        paper_data.get('sub_category'),
        paper_data.get('source')
    )
    
    try:
        cursor.execute(insert_paper, paper_values)
        
        # Process creators (authors)
        if 'creators' in paper_data and paper_data['creators']:
            creators_list = [creator.strip() for creator in paper_data['creators'].split(';') if creator.strip()]
            if len(creators_list) == 1 and ';' not in paper_data['creators']:
                creators_list = [paper_data['creators'].strip()]
                
            for creator_name in creators_list:
                # Insert creator if not exists
                cursor.execute("INSERT IGNORE INTO creators (name) VALUES (%s)", (creator_name,))
                cursor.execute("SELECT id FROM creators WHERE name = %s", (creator_name,))
                creator_id = cursor.fetchone()[0]
                
                # Link creator to paper
                cursor.execute(
                    "INSERT IGNORE INTO paper_creators (paper_id, creator_id) VALUES (%s, %s)",
                    (paper_data.get('id'), creator_id)
                )
        
        # Process contributors (advisors, committee members)
        if 'contributors' in paper_data and paper_data['contributors']:
            contributors_list = [contrib.strip() for contrib in paper_data['contributors'].split(';') if contrib.strip()]
            
            for contributor_info in contributors_list:
                if '(' in contributor_info and ')' in contributor_info:
                    # Extract name and role
                    name_part, role_part = contributor_info.split('(', 1)
                    contributor_name = name_part.strip()
                    role = role_part.split(')', 1)[0].strip()
                else:
                    contributor_name = contributor_info
                    role = "Unknown"
                
                # Insert contributor if not exists
                cursor.execute("INSERT IGNORE INTO contributors (name) VALUES (%s)", (contributor_name,))
                cursor.execute("SELECT id FROM contributors WHERE name = %s", (contributor_name,))
                contributor_id = cursor.fetchone()[0]
                
                # Link contributor to paper with role
                cursor.execute(
                    "INSERT IGNORE INTO paper_contributors (paper_id, contributor_id, role) VALUES (%s, %s, %s)",
                    (paper_data.get('id'), contributor_id, role)
                )
        
        # Process subjects/keywords
        if 'subjects' in paper_data and paper_data['subjects']:
            subjects_list = [subject.strip() for subject in paper_data['subjects'].split(';') if subject.strip()]
            if len(subjects_list) == 1 and ';' not in paper_data['subjects']:
                subjects_list = [paper_data['subjects'].strip()]
                
            for subject_name in subjects_list:
                # Insert subject if not exists
                cursor.execute("INSERT IGNORE INTO subjects (name) VALUES (%s)", (subject_name,))
                cursor.execute("SELECT id FROM subjects WHERE name = %s", (subject_name,))
                subject_id = cursor.fetchone()[0]
                
                # Link subject to paper
                cursor.execute(
                    "INSERT IGNORE INTO paper_subjects (paper_id, subject_id) VALUES (%s, %s)",
                    (paper_data.get('id'), subject_id)
                )
        
        database.commit()
        return True
    except Exception as e:
        print(f"Error inserting paper {paper_data.get('id')}: {e}")
        database.rollback()
        return False

# Example usage with a single file
# json_file_path = "../../data/dewey_thesis/9000.json"
# result = insert_paper_from_json(json_file_path)
# print(f"Inserted paper from {json_file_path}: {result}")

In [65]:
# Bulk insert papers from the dewey_thesis folder
def insert_all_papers_from_folder(folder_path):
    # Import tqdm for progress bar
    
    successful = 0
    failed = 0
    
    # Get list of JSON files first
    json_files = [filename for filename in os.listdir(folder_path) if filename.endswith('.json')]
    
    # Use tqdm to show progress
    for filename in tqdm(json_files, desc="Importing papers", unit="file"):
        file_path = os.path.join(folder_path, filename)
        try:
            if insert_paper_from_json(file_path):
                successful += 1
            else:
                failed += 1
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            failed += 1
    
    return successful, failed

# Uncomment to run the bulk import
# Note: This may take a while depending on the number of files
dewey_thesis_folder = "../../data/dewey_thesis"
successful, failed = insert_all_papers_from_folder(dewey_thesis_folder)
print(f"Bulk import complete: {successful} papers imported successfully, {failed} failed")

Importing papers: 100%|██████████| 41597/41597 [13:21<00:00, 51.90file/s]

Bulk import complete: 41597 papers imported successfully, 0 failed


In [66]:
# Query to test and verify the database structure
def test_database_structure():
    # Check the number of papers
    cursor.execute("SELECT COUNT(*) FROM dewey_papers")
    paper_count = cursor.fetchone()[0]
    print(f"Total papers in database: {paper_count}")
    
    # Sample query to get a paper with its creators and subjects
    query = """
    SELECT 
        p.id, p.title, p.abstract, 
        GROUP_CONCAT(DISTINCT c.name SEPARATOR '; ') as creators,
        GROUP_CONCAT(DISTINCT s.name SEPARATOR '; ') as subjects
    FROM 
        dewey_papers p
    LEFT JOIN 
        paper_creators pc ON p.id = pc.paper_id
    LEFT JOIN 
        creators c ON pc.creator_id = c.id
    LEFT JOIN 
        paper_subjects ps ON p.id = ps.paper_id
    LEFT JOIN 
        subjects s ON ps.subject_id = s.id
    GROUP BY 
        p.id
    LIMIT 1
    """
    
    cursor.execute(query)
    result = cursor.fetchone()
    
    if result:
        print("\nSample paper:")
        print(f"ID: {result[0]}")
        print(f"Title: {result[1]}")
        print(f"Abstract: {result[2][:100]}...")
        print(f"Creators: {result[3]}")
        print(f"Subjects: {result[4]}")
    else:
        print("No papers found in database")

# Run the test
test_database_structure()

Total papers in database: 41597

Sample paper:
ID: 1000
Title: The Indonesian subtitles of the English utterances spoken by the characters in American Beauty
Abstract: This thesis is particularly about the translation done on the subtitles of the movie entitled Americ...
Creators: BERLIN RAHAYU
Subjects: ENGLISH LANGUAGE-SLANG; ENGLISH LANGUAGE-TRANSLATING INTO INDONESIAN


In [67]:
# Close the database connection when done
# Uncomment when you're finished with the database
cursor.close()
database.close()
print("Database connection closed")

Database connection closed
